In [49]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data.dataset import random_split

torch.manual_seed(265)                  # Sets the randomness
torch.set_default_dtype(torch.double)   # Sets the default datatype for tensors

In [27]:
transform = transforms.Compose([
    transforms.ToTensor(),                          #converts to tensor
    transforms.Lambda(lambda x: torch.flatten(x))   #Flattens the tensor
])

# Loads the cifar10 training and test sets. It downloads it into the data folder and is transformed into a tenser using the transfrom.toTensor.
full_training_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
full_test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [53]:
# Filters out the nonrelevant labels. Keeps the airplane and birds
training_set = Subset(full_training_set, [i for i, (_, label) in enumerate(full_training_set) if label in [0,2]])
test_set = Subset(full_test_set, [i for i, (_, label) in enumerate(full_test_set) if label in [0,2]])

In [60]:
# Splits training set into training and validation set
training_size = int(len(training_set)*0.9)
validation_size = len(training_set) - training_size

training_set, validation_set = random_split(training_set, [training_size, validation_size])

In [30]:
# MLP class for 3.1.2

# CRITERIA:
#   * The input dimension is 3072 (= 32*32*3) and the output dimension is 2 (for the 2 classes).
#   * The hidden layers have respectively 512, 128 and 32 hidden units.
#   * All activation functions are ReLU. The last layer has no activation function since the cross-entropy loss already includes a softmax activation function.

# Layout: [3072, 512, 128, 32, 2]

class MyMLP(nn.Module):

    def __init__(self):
        super().__init__() # Initializes the nn.module

        # Defines the network using seqential.
        self.model = nn.Sequential(
            nn.Linear(in_features=3072, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=2)
        )

    def forward(self, input): #Passes the input trough the network and returns the output
        return self.model(input)

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Sets the device

In [68]:

def train(n_epochs, optimizer, model, loss_fn, train_loader):

    model.to(device)
    model.train()

    for i in range(n_epochs):
        total_loss = 0.0
        batch_amount = len(train_loader)

        for x, y_true  in train_loader:
            x, y_true = x.to(device, dtype=torch.double), y_true.to(device)

            optimizer.zero_grad()           # Resets the gradients

            y_pred = model(x)               # Forward passes the batch

            loss = loss_fn(y_pred, y_true)  # calcs the loss
            total_loss += loss.item()       # Adds the loss to the total

            loss.backward()                 # Backward propogation
            optimizer.step()                # Updates the weights and biases

        print("Train ||| Epoch: ",(i+1), " of ", n_epochs, "| Average Loss: ", total_loss / batch_amount)


In [66]:
def train_manual_update(n_epochs, lr, model, loss_fn, train_loader, weight_decay, momentum):

    model.to(device)
    model.train()

    for i in range(n_epochs):
        total_loss = 0.0
        batch_amount = len(train_loader)

        for x, y_true  in train_loader:
            x, y_true = x.to(device, dtype=torch.double), y_true.to(device)
            
            model.zero_grad()           # Resets the gradients

            y_pred = model(x)               # Forward passes the batch

            loss = loss_fn(y_pred, y_true)  # calcs the loss
            total_loss += loss.item()       # Adds the loss to the total

            loss.backward()                 # Backward propogation
            
            with torch.no_grad():           #Manually uses gradient decent to update each oarameter
                for p in model.parameters():
                    p = p - lr*p.grad       # Equation (2) in project pdf.

        print("Train_manual_update ||| Epoch: ",(i+1), " of ", n_epochs, "| Average Loss: ", total_loss / batch_amount)

In [69]:
torch.manual_seed(265)

model = MyMLP()

lr = 0.1

n_epochs = 10
optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()
train_loader = DataLoader(training_set, 32)

train(n_epochs, optimizer, model, loss_fn, train_loader)
train_manual_update(n_epochs, lr, model, loss_fn, train_loader)


IndexError: Target 2 is out of bounds.